# Learning to work with the data

Import relevant packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from plotting_functions_BES import *
import matplotlib.path as mplPath

Read $D_\alpha$ and density data

In [2]:
# for density, Dalpha and upper tangential Dalpha
dalpha_from_file = xr.open_dataarray('Data/shot29378_dalpha.nc')

dalpha_time = np.asarray(dalpha_from_file.coords['time'])
dalpha_data = np.asarray(dalpha_from_file)

density_from_file = xr.open_dataarray('Data/shot29378_density.nc')

density_time = np.asarray(density_from_file.coords['time'])
density_data = np.asarray(density_from_file)

Read equilibria data

In [3]:
# for equilibria
equilib_from_file = xr.open_dataarray('Data/shot29378_equilibria.nc')

equilib_time = np.asarray(equilib_from_file.coords['time'])
equilib_R = np.asarray(equilib_from_file.coords['R'])
equilib_Z = np.asarray(equilib_from_file.coords['Z'])
equilib_psi = np.asarray(equilib_from_file) # Contains 135 length-65 arrays [0:134][0:64]

Read BES data

In [4]:
# for BES data
# (R, z) locations for the BES channels (view location)
apdpos = np.asarray(xr.open_dataarray('Data/shot29378_apdpos.nc'))

fluct_data_from_file = xr.open_dataarray('Data/shot29378_LH_fluct_data.nc')

bes_time = np.asarray(fluct_data_from_file.coords['time'])
fluct_data = np.asarray(fluct_data_from_file)
# fluct_data has 4 rows, 8 channels (mapped to apdpos)

Contour plot of flux surfaces from equilibrium data

In [5]:
time_idx = 50 # 0 to 132
"""plt.contour(equilib_R, equilib_Z, equilib_psi[time_idx], np.linspace(0, 1, 21))
plt.title("Time = " + str(equilib_time[time_idx]))
plt.xlabel("$R$ / m")
plt.ylabel("$Z$ / m")
plt.colorbar()
plt.show()"""

'plt.contour(equilib_R, equilib_Z, equilib_psi[time_idx], np.linspace(0, 1, 21))\nplt.title("Time = " + str(equilib_time[time_idx]))\nplt.xlabel("$R$ / m")\nplt.ylabel("$Z$ / m")\nplt.colorbar()\nplt.show()'

Generate plot of fluctation against time for a given BES channel from fluctuation data

In [6]:
channel_id = 5 # 0 to 31
channel_coords = apdpos[channel_id] # Get corresponding (R,Z) coordinates

#plot_bes_fluctuations(29378, bes_time, fluct_data, [0.36,0.54], [0,1], "fluct_plot")

Generate grid of channels and fluctuation value at a given time

In [7]:
time_idx=992499 # 0 to 992499
channels = [] # 32x1 array of the 32 channels
for channel in fluct_data:
    channels.append(channel[time_idx])
channels = np.reshape(channels, (4,8)) # Convert 32x1 array to 8x4, presumed to be correct
"""plt.matshow(channels)
plt.title("Time =" + str(bes_time[time_idx]))
plt.xlabel("Channel")
plt.ylabel("Channel")
plt.colorbar()"""

'plt.matshow(channels)\nplt.title("Time =" + str(bes_time[time_idx]))\nplt.xlabel("Channel")\nplt.ylabel("Channel")\nplt.colorbar()'

In [8]:
#plot_bes_locs(29378, apdpos, equilib_time[50], equilib_R, equilib_Z, equilib_psi[50])

Plotting $D_\alpha$ data (density) to identify timeslices of interest where there appears to be repetitive behaviour.

In [9]:
"""figure, axes = plt.subplots(2, 1, sharex=True, 
                                figsize=(15, 6))
axes[0].plot(dalpha_time, dalpha_data, 'k', linewidth=0.8)
axes[1].plot(density_time, density_data, 'k', linewidth=0.8)
axes[1].tick_params(axis='x', labelsize=24)
axes[1].set_xlabel('time [s]', fontsize=26)
axes[1].set_xlim([dalpha_time[0], dalpha_time[-1]])
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()
plt.close()"""

"figure, axes = plt.subplots(2, 1, sharex=True, \n                                figsize=(15, 6))\naxes[0].plot(dalpha_time, dalpha_data, 'k', linewidth=0.8)\naxes[1].plot(density_time, density_data, 'k', linewidth=0.8)\naxes[1].tick_params(axis='x', labelsize=24)\naxes[1].set_xlabel('time [s]', fontsize=26)\naxes[1].set_xlim([dalpha_time[0], dalpha_time[-1]])\nplt.subplots_adjust(wspace=0, hspace=0)\nplt.show()\nplt.close()"

In [10]:
"""plot_dalpha(29378, dalpha_time, dalpha_data, [0.16,0.24], "region_1")
plot_density(29378, density_time, density_data, [0.16,0.24], "region_1")

plot_dalpha(29378, dalpha_time, dalpha_data, [0.36,0.54], "region_2")
plot_density(29378, density_time, density_data, [0.36,0.54], "region_2")

plot_dalpha(29378, dalpha_time, dalpha_data, [0.54,0.68], "region_3")
plot_density(29378, density_time, density_data, [0.54,0.68], "region_3")"""

'plot_dalpha(29378, dalpha_time, dalpha_data, [0.16,0.24], "region_1")\nplot_density(29378, density_time, density_data, [0.16,0.24], "region_1")\n\nplot_dalpha(29378, dalpha_time, dalpha_data, [0.36,0.54], "region_2")\nplot_density(29378, density_time, density_data, [0.36,0.54], "region_2")\n\nplot_dalpha(29378, dalpha_time, dalpha_data, [0.54,0.68], "region_3")\nplot_density(29378, density_time, density_data, [0.54,0.68], "region_3")'

How uniformly spaces is <code>bes_time</code>?

In [11]:
time_diffs = np.diff(bes_time)
print("Mean =", np.mean(time_diffs), "s\nStandard Deviation =", np.std(time_diffs), "s")

Mean = 4.99999999992207e-07 s
Standard Deviation = 2.42898975271345e-17 s


The step appears to be highly uniform so the FFT provided by <code>scipy</code> is reasonable to use.

Plot full $D_\alpha$ data, highlighting the regions of interest and storing their data in arrays

In [12]:
regions = [[0.16,0.24], [0.36,0.54], [0.54,0.68]]
region_data = []
region_times = []

for i in range(3):
    idx1 = (np.abs(dalpha_time - regions[i][0])).argmin()
    idx2 = (np.abs(dalpha_time - regions[i][1])).argmin()
    region_data.append(dalpha_data[idx1:idx2+1]) # +1 to include end
    region_times.append(dalpha_time[idx1:idx2+1])

"""plt.plot(dalpha_time, dalpha_data, 'k', linewidth=0.5)
plt.plot(region_times[0], region_data[0], "r", linewidth=0.5)
plt.plot(region_times[1], region_data[1], "g", linewidth=0.5)
plt.plot(region_times[2], region_data[2], "b", linewidth=0.5)

plt.xlabel("Time [s]")
plt.ylabel(r"Density [m$^{-3}$]")
#plt.savefig("Plots/Full_Dalpha_plot.pdf", format="pdf", bbox_inches="tight")
plt.show()"""

'plt.plot(dalpha_time, dalpha_data, \'k\', linewidth=0.5)\nplt.plot(region_times[0], region_data[0], "r", linewidth=0.5)\nplt.plot(region_times[1], region_data[1], "g", linewidth=0.5)\nplt.plot(region_times[2], region_data[2], "b", linewidth=0.5)\n\nplt.xlabel("Time [s]")\nplt.ylabel(r"Density [m$^{-3}$]")\n#plt.savefig("Plots/Full_Dalpha_plot.pdf", format="pdf", bbox_inches="tight")\nplt.show()'

Plot $D_\alpha$ data for each region of interest.

In [13]:
"""figure, axes = plt.subplots(3, 1, sharex=False, figsize=(15, 9))

for i in range(3):
    idx1 = (np.abs(dalpha_time - regions[i][0])).argmin()
    idx2 = (np.abs(dalpha_time - regions[i][1])).argmin()
    axes[i].plot(dalpha_time[idx1:idx2], dalpha_data[idx1:idx2], 'k', linewidth=0.8)
    axes[i].tick_params(axis='x', labelsize=24)
    axes[i].tick_params(axis='y', labelsize=20)
    axes[i].set_ylabel(r"Region " + str(i+1) + "\nDensity [m$^{-3}$]", fontsize=20)
    axes[i].locator_params(axis='y', nbins=3)    

axes[-1].set_xlabel('Time [s]', fontsize=26)
plt.subplots_adjust(hspace=0.5)
#plt.savefig("Plots/Dalpha_region_plots.pdf", format="pdf", bbox_inches="tight")
plt.show()"""

'figure, axes = plt.subplots(3, 1, sharex=False, figsize=(15, 9))\n\nfor i in range(3):\n    idx1 = (np.abs(dalpha_time - regions[i][0])).argmin()\n    idx2 = (np.abs(dalpha_time - regions[i][1])).argmin()\n    axes[i].plot(dalpha_time[idx1:idx2], dalpha_data[idx1:idx2], \'k\', linewidth=0.8)\n    axes[i].tick_params(axis=\'x\', labelsize=24)\n    axes[i].tick_params(axis=\'y\', labelsize=20)\n    axes[i].set_ylabel(r"Region " + str(i+1) + "\nDensity [m$^{-3}$]", fontsize=20)\n    axes[i].locator_params(axis=\'y\', nbins=3)    \n\naxes[-1].set_xlabel(\'Time [s]\', fontsize=26)\nplt.subplots_adjust(hspace=0.5)\n#plt.savefig("Plots/Dalpha_region_plots.pdf", format="pdf", bbox_inches="tight")\nplt.show()'

In [14]:
idx1 = (np.abs(equilib_time - regions[2][0])).argmin()
idx2 = (np.abs(equilib_time - regions[2][1])).argmin()
#for i in range(idx1,idx2):
#    plot_bes_locs(29378, apdpos, equilib_time[i], equilib_R, equilib_Z, equilib_psi[i])
#print(sep_temp(equilib_R, equilib_Z, equilib_psi[idx1], apdpos))
#plot_bes_locs(29378, apdpos, equilib_time[idx1], equilib_R, equilib_Z, equilib_psi[idx1])

Region 1: col 1. Region 2: cols 1, 2. Region 3: cols 1, 2, 3.

Function to determine if a given point lies inside the separatrix. Not visually tested on all extreme cases but seems to work. The warning 'UserWarning: No contour levels were found within the data range.' - this just means the contour is too far to the left at that time. A problem in some special cases e.g. at end of region 3. Over the times that will be useful for analysis there should be a contour in the vicinity so this warning shouldn't occur.

In [214]:
def inside_sep(equilib_R, equilib_Z, equilib_psi_t, apdpos):
    contains = []
    
    # Only looking for points in the relevant z and right of array's left-most point
    # Add on bits to extend contour far enough
    zmin = np.abs(np.min(apdpos[:,1])-equilib_Z-0.1).argmin()
    zmax = np.abs(np.max(apdpos[:,1])-equilib_Z+0.1).argmin()
    rmin = np.abs(np.min(apdpos[:,0])-equilib_R-0.1).argmin()
    
    # Get path of separatrix
    CS = plt.contour(equilib_R[rmin:], equilib_Z[zmin:zmax], equilib_psi_t[zmin:zmax,rmin:], [1.0])
    plt.close()
    path = mplPath.Path(CS.allsegs[0][0])
    
    # Extrapolate to get more coordinates
    verts = path.vertices
    verts_expanded = []
    idx = 0
    for i in verts[:-1]:
        for r in np.linspace(verts[idx,0],verts[idx+1,0],100): # Get lots of points
            verts_expanded.append([r, np.interp(r, verts[idx:idx+2,0], verts[idx:idx+2,1])])
        idx += 1
    verts_expanded = np.asarray(verts_expanded)
    
    # Find r-coord of point closest in z to channel and test if it's further out
    for ch in apdpos:
        closest_r = verts_expanded[np.abs(verts_expanded[:,1]-ch[1]).argmin(),0]
        if closest_r > ch[0]:
            contains.append(True)
            #plt.scatter(ch[0], ch[1], color="g")
        else:
            contains.append(False)
            #plt.scatter(ch[0], ch[1], color="r")
            
    return np.asarray(contains)

In [215]:
#inside_sep(equilib_R, equilib_Z, equilib_psi[116], apdpos)

array([ True,  True,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True, False, False, False,  True,  True,
        True,  True, False, False, False, False,  True,  True,  True,
        True, False, False, False, False])

We want a to be able to find out which columns are always inside separatrix for each region, and how much of each row is always inside it.

In [211]:
region_times_equilib = []
for region in regions:
    idx1 = (np.abs(equilib_time - region[0])).argmin()
    idx2 = (np.abs(equilib_time - region[1])).argmin()
    region_times_equilib.append(np.arange(idx1,idx2+1,1)) # Need +1 to include end point
region_times_equilib[2] = region_times_equilib[2][:-1] # Handle special case

Function to determine which channels are always within separatrix for a given region. A refined version could be something like where is True the case most of the time. Avoid excluding a column because of contour at one point in time.

In [271]:
def inside_sep_all_time(region):
    region = region -1
    all_time = [True for i in range(32)]
    for time in region_times_equilib[region]:
        this_time = inside_sep(equilib_R, equilib_Z, equilib_psi[time], apdpos)
        for ch in range(32):
            if this_time[ch] == False:
                all_time[ch] = False
    all_time = np.asarray(all_time)
    return all_time

In [279]:
for region in range(1,4):
    print(inside_sep_all_time(region).reshape(4,8), end="\n\n")

[[ True  True False False False False False False]
 [ True  True False False False False False False]
 [ True  True False False False False False False]
 [ True  True False False False False False False]]

[[ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True False False False False False False]
 [ True  True False False False False False False]]

[[ True  True  True  True False False False False]
 [ True  True  True  True False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False False False False]]



Function to say whether or not a given column is always inside separatrix for a given region.

In [274]:
def check_col_good(region, col): # region and col start from 1
    all_time = inside_sep_all_time(region)
    region = region-1
    col = col-1
    
    this_col = [all_time[ch] for ch in np.arange(col,32,8)]
    if False not in this_col:
        return True
    else:
        return False

In [276]:
"""for region in range(1,4):
    for col in range(1,9):
        print(check_col_good(region,col))"""

'for region in range(1,4):\n    for col in range(1,9):\n        print(check_col_good(region,col))'

This says first 2 cols are good for regions 1 & 2, and first 3 cols for region 3. Will include some cols where channel is juuuust inside.

Function to say how much of a row is always inside separatrix for a given region.

In [304]:
def check_row_lim(region, row): # region and col start from 1
    all_time = inside_sep_all_time(region)
    region = region-1
    row = row-1
    
    chans = [ch for ch in range(8*row,8*row+8)]
    this_row = [all_time[ch] for ch in chans]
    if False in this_row:
        return this_row.index(False)-1 # Return one before False, i.e. the last True
    else:
        return chans[-1] # All channels always inside. Return index of the end

In [301]:
"""for region in range(1,4):
    for row in range(1,5):
        print("Region", region, ", row", row, check_row_lim(region,row))
    print("")"""

Region 1 , row 1 1
Region 1 , row 2 1
Region 1 , row 3 1
Region 1 , row 4 1

Region 2 , row 1 2
Region 2 , row 2 2
Region 2 , row 3 1
Region 2 , row 4 1

Region 3 , row 1 3
Region 3 , row 2 3
Region 3 , row 3 2
Region 3 , row 4 2



This says: for region 1, the first 2 channels of each row are always inside. For region 2 it's the same, but rows 1&2 have 3 inside. For region 3, all have first 3 with rows 1&2 also having another, for 4 total.

In [17]:
#plot_bes_fluct_spectrogram(29378, bes_time, fluct_data, [i for i in range(32)], [bes_time[0], bes_time[-1]], 8, "full", threshold=0.54)